In [2]:
from dipy.reconst import mapmri
# from dipy.viz import window, actor
from dipy.data import fetch_cfin_multib, read_cfin_dwi, get_sphere
from dipy.core.gradients import gradient_table
from dipy.io import read_bvals_bvecs

import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import os

from mpl_toolkits.axes_grid1 import make_axes_locatable

In [3]:
patient_id = 11

dwi_big_delta = 0.0334 # scanner default
dwi_small_delta = 0.0322 # scanner default

#dwi_big_delta = 0.045  # desired


In [4]:
data_dir = "/user/pfilipia/home/inria/chu_nice_inria/patients_dmri"

patient_dir = os.path.join(
    data_dir, "patient%02d/bids/sub-patient%02d/ses-presurgical" % (patient_id, patient_id)
)

def get_dwi_file(patient_id):
    return os.path.join(
        patient_dir, "proc/sub-patient%02d_ses-presurgical_dwi.nii.gz" % patient_id
    )

def get_bval_file(patient_id):
    return os.path.join(
        patient_dir, "proc/sub-patient%02d_ses-presurgical_dwi.bval" % patient_id
    )

def get_bvec_file(patient_id):
    return os.path.join(
        patient_dir, "proc/sub-patient%02d_ses-presurgical_dwi.bvec" % patient_id
    )

def get_mask_file(patient_id):
    return os.path.join(
        patient_dir, "connectivity/tck_streamlines/roi_mask.nii.gz"
    )
#     return os.path.join(
#         patient_dir, "connectivity/streamlines_shortest/roi_mask.nii.gz"
#     )

def get_microstructure_indices_dir(patient_id):
    indices_dir = os.path.join(
        patient_dir, "connectivity/tck_streamlines/indices"
    )
#     indices_dir = os.path.join(
#         patient_dir, "connectivity/streamlines_shortest/indices"
#     )
    if not os.path.isdir(indices_dir):
        os.mkdir(indices_dir)
    return indices_dir

def get_microstructure_index_file(patient_id, index_name):
    return os.path.join(
        get_microstructure_indices_dir(patient_id), "patient%02d_roi_%s.nii.gz" % (patient_id, index_name)
    )


In [ ]:
dwi_img = nib.load(get_dwi_file(patient_id))
dwi_data = dwi_img.get_data()


In [ ]:
mask_img = nib.load(get_mask_file(patient_id))
mask_data = mask_img.get_data()


In [ ]:
bvals, bvecs = read_bvals_bvecs(get_bval_file(patient_id), get_bvec_file(patient_id))
dwi_gtab = gradient_table(bvals, bvecs, big_delta=dwi_big_delta, small_delta=dwi_small_delta)


In [ ]:
radial_order = 6
map_model_laplacian_aniso = mapmri.MapmriModel(
    dwi_gtab, radial_order=radial_order,
    laplacian_regularization=True, laplacian_weighting=.2,
    bval_threshold=2000
)

In [ ]:
mapfit_laplacian_aniso = map_model_laplacian_aniso.fit(dwi_data, mask=mask_data)

In [ ]:
microstructure_indices = {
    'rtop'   : np.array(mapfit_laplacian_aniso.rtop(), dtype=float),
    'rtap'   : np.array(mapfit_laplacian_aniso.rtap(), dtype=float),
    'rtpp'   : np.array(mapfit_laplacian_aniso.rtpp(), dtype=float),
    'msd'    : np.array(mapfit_laplacian_aniso.msd(), dtype=float),
    'qiv'    : np.array(mapfit_laplacian_aniso.qiv(), dtype=float),
    'ng'     : np.array(mapfit_laplacian_aniso.ng(), dtype=float),
    'ng_perp': np.array(mapfit_laplacian_aniso.ng_perpendicular(), dtype=float),
    'ng_par' : np.array(mapfit_laplacian_aniso.ng_parallel(), dtype=float)
}

In [ ]:
for idx in microstructure_indices:
    idx_img = nib.Nifti1Image(microstructure_indices[idx], dwi_img.affine, dwi_img.header)
    nib.save(idx_img, get_microstructure_index_file(patient_id, idx))
